In [1]:
from sqlalchemy import create_engine, Column, String, Integer, func, event, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.orm import sessionmaker
from geoalchemy2 import Geometry 
from tqdm import tqdm
from shapely.wkt import dumps

import orjson


In [2]:
%%time
with open('13_266069_040_003 L02 PAS.json', 'r') as file:
    data = orjson.loads(file.read())

import shapely
from shapely.geometry import shape

CPU times: user 9.39 s, sys: 1.58 s, total: 11 s
Wall time: 12.2 s


In [3]:
data[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[45837, 20092],
    [45836, 20093],
    [45835, 20094],
    [45834, 20095],
    [45833, 20096],
    [45832, 20096],
    [45831, 20096],
    [45830, 20096],
    [45829, 20096],
    [45828, 20096],
    [45827, 20096],
    [45826, 20096],
    [45825, 20096],
    [45824, 20096],
    [45823, 20096],
    [45822, 20096],
    [45821, 20096],
    [45820, 20097],
    [45819, 20098],
    [45818, 20099],
    [45817, 20100],
    [45816, 20100],
    [45815, 20100],
    [45814, 20100],
    [45813, 20100],
    [45812, 20101],
    [45811, 20102],
    [45810, 20103],
    [45809, 20104],
    [45808, 20104],
    [45807, 20104],
    [45806, 20104],
    [45805, 20104],
    [45804, 20105],
    [45803, 20106],
    [45802, 20107],
    [45801, 20108],
    [45800, 20109],
    [45799, 20110],
    [45798, 20111],
    [45797, 20112],
    [45796, 20113],
    [45795, 20114],
    [45794, 20115],
    [45793, 20116],
    [45792, 20117],
    [45791, 

In [4]:
# Create a base class for our declarative mapping
Base = declarative_base()

# Define your SQLAlchemy model
class GeometryModel(Base):
    __tablename__ = 'geometries'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    geom = Column(Geometry('POLYGON'))

/tmp/ipykernel_18134/2237727833.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [5]:
# Connect to the Spatialite database
db_path = 'sqlite:////tmp/blog/spatialite_orm.db'
engine = create_engine(db_path, echo=False)  # Set echo=True to see SQL commands being executed

In [6]:
# Initialize Spatialite extension
@event.listens_for(engine, "connect")
def connect(dbapi_connection, connection_record):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    dbapi_connection.execute('SELECT InitSpatialMetaData(1);')

In [7]:
# Create the table
Base.metadata.create_all(engine)

# Start a session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
from tqdm import tqdm

In [9]:
%%time
for _ in range(12):
    batch_size=5_000
    polygons=[]
    with  engine.connect() as conn:
        for geojson in tqdm(data):
            name = geojson["properties"]["classification"]["name"]
            shapely_geom = shape(geojson["geometry"])
            
            polygons.append(GeometryModel(name=name, geom=shapely_geom.wkt))
        
            if len(polygons) == batch_size:
                session.bulk_save_objects(polygons)
                session.commit()
                polygons.clear()  # Clear the list for the next batch
        
        # Insert any remaining records that didn't fit into the final batch
        if polygons:
            session.bulk_save_objects(polygons)
            session.commit()
session.close()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 88605/88605 [00:36<00:00, 2456.44it/s]


CPU times: user 7min 5s, sys: 13.5 s, total: 7min 19s
Wall time: 7min 30s


In [10]:
%%time
#lets make sure insert worked as expected
with  engine.connect() as conn:
    res=conn.execute(text("select count(geom) from geometries"))
    nresults=res.fetchall()
    print(nresults)

[(1063260,)]
CPU times: user 216 ms, sys: 1.15 s, total: 1.37 s
Wall time: 1.36 s


In [13]:
%%time
with  engine.connect() as conn:
    res=conn.execute(text("select AsGeoJSON(ST_Centroid(geom)) as centroid from geometries limit 1000"))
    centroids=res.fetchall()

CPU times: user 19.1 ms, sys: 368 µs, total: 19.5 ms
Wall time: 17 ms


In [14]:
centroids[0:100]

[('{"type":"Point","coordinates":[45862.13292750414,20242.07223759453]}',),
 ('{"type":"Point","coordinates":[45812.26248864668,20306.06145927944]}',),
 ('{"type":"Point","coordinates":[45854.85213032581,20261.92932330827]}',),
 ('{"type":"Point","coordinates":[45901.874369386,20236.45970222714]}',),
 ('{"type":"Point","coordinates":[46006.16937669376,20198.66260162601]}',),
 ('{"type":"Point","coordinates":[45858.96278317151,20359.42055016181]}',),
 ('{"type":"Point","coordinates":[45792.37442014578,20346.95361166335]}',),
 ('{"type":"Point","coordinates":[45750.40160878947,20334.95173631548]}',),
 ('{"type":"Point","coordinates":[45954.05971937029,20293.65349075975]}',),
 ('{"type":"Point","coordinates":[45759.33590138675,20285.43374422187]}',),
 ('{"type":"Point","coordinates":[45982.3797250859,20206.2147766323]}',),
 ('{"type":"Point","coordinates":[45981.7408456021,20166.57002642506]}',),
 ('{"type":"Point","coordinates":[45803.70650616221,20158.2388936658]}',),
 ('{"type":"Point"